# Function for cutting too big and too small samples of jobs

In [170]:
import pandas as pd

## The processed dataset
# df = pd.read_csv('datasets\processed_data.csv')

## The original one
df = pd.read_parquet('job_title_dataset.parquet')

In [171]:
df_jobs_count = df.JobTitle.value_counts().to_frame(name='amount')
df_jobs_count.head()

,amount
Software Engineer,342
Physical Therapist Assistant (PTA),200
Loan Officer,116
Content Strategist,43
Wholesaler,40


In [172]:
def reshape_df(df, the_min_amount_of_rows = 10, the_max_amount_of_rows = 100):
    list_of_two_parts = [pd.DataFrame(columns=df.columns)] * 2    ## In the end we want to concat the two part
    
    df_jobs_count = df.JobTitle.value_counts().to_frame(name='amount')
    
    ## Droping every row under minimum amount of observations
    df_jobs_to_drop = df_jobs_count.query(f'amount<{the_min_amount_of_rows}')
    list_jobs_to_drop = list(df_jobs_to_drop.index)
    df = df[-df.JobTitle.isin(list_jobs_to_drop)]
    
    ## Creating the first part of list, the one we don't touch
    df_jobs_no_problem = df_jobs_count.query(f'amount>={the_min_amount_of_rows} & amount<={the_max_amount_of_rows}')
    list_of_two_parts[0] = df[df.JobTitle.isin(df_jobs_no_problem.index)]
    
    ## Creating the the second part of the list, where we resample the jobs with too many rows
    df_jobs_to_resample = df_jobs_count.query(f'amount>{the_max_amount_of_rows}')
    list_jobs_to_resample = list(df_jobs_to_resample.index)
    df_for_resampling = df[df.JobTitle.isin(list_jobs_to_resample)]
    
    list_of_subsets = [pd.DataFrame(columns=df.columns)] * len(list_jobs_to_resample)
    for i in range(len(list_jobs_to_resample)):
        list_of_subsets[i] = df_for_resampling[df_for_resampling.JobTitle == list_jobs_to_resample[i]].sample(the_max_amount_of_rows)
    if len(list_of_subsets) !=0:
           list_of_two_parts[1] = pd.concat(list_of_subsets)
    
    return pd.concat(list_of_two_parts)

In [173]:
df = reshape_df(df, the_min_amount_of_rows = 10, the_max_amount_of_rows = 100)

In [174]:
df_jobs_count = df.JobTitle.value_counts().to_frame(name='amount')
df_jobs_count

,amount
Loan Officer,100
Physical Therapist Assistant (PTA),100
Software Engineer,100
Content Strategist,43
Wholesaler,40
Automotive Service Manager,34
Growth Strategy Manager,30
Neonatal Nurse,26
Medical Laboratory Technician (MLT),25
Human Resources Assistant,24


In [100]:
df_jobs_no_problem = df_jobs_count.query('amount>=10 & amount<=100')
# list_jobs_no_problem = list(df_jobs_no_problem.index)
# list_jobs_no_problem
df0 = df[df.JobTitle.isin(df_jobs_no_problem.index)]
df0

,amount
Content Strategist,43
Wholesaler,40
Automotive Service Manager,34
Growth Strategy Manager,30
Neonatal Nurse,26
Medical Laboratory Technician (MLT),25
Human Resources Assistant,24
Pediatric Anesthesiologist,21
Data Science Director,19
Tax Analyst,16


In [17]:
# df_jobs_to_drop = df_jobs_count.query('amount<10')
# df_jobs_to_drop.sample(5)

In [18]:
df_jobs_to_resample = df_jobs_count.query('amount>100')
df_jobs_to_resample

,amount
Software Engineer,342
Physical Therapist Assistant (PTA),200
Loan Officer,116


In [19]:
list_jobs_to_resample = list(df_jobs_to_resample.index)
type(df_jobs_to_resample.index)

pandas.core.indexes.base.Index

In [ ]:
df.JobTitle[0] in list_jobs_to_resample

In [11]:
df_for_resampling = df[df.JobTitle.isin(df_jobs_to_resample.index)]
len(df_for_resampling)

658

In [21]:
df_without_huge_jobs = df[-df.JobTitle.isin(df_jobs_to_resample.index)]
df_without_huge_jobs.JobTitle.value_counts().to_frame(name='amount').head()

,amount
Content Strategist,43
Wholesaler,40
Automotive Service Manager,34
Growth Strategy Manager,30
Neonatal Nurse,26


In [45]:
list_of_subsets = [None] * len(list_jobs_to_resample)

In [50]:
for i in range(len(list_jobs_to_resample)):
    list_of_subsets[i] = df_for_resampling[df_for_resampling.JobTitle == list_jobs_to_resample[i]].sample(100)
df_new = pd.concat(list_of_subsets)
df_new

,Description,JobTitle
181,"['software', 'engineer', 'personalization', 'e...",Software Engineer
311,"['hp', 'believe', 'power', 'ideas', 'vision', ...",Software Engineer
326,"['become', 'part', 'nike', 'inc', 'teamnike', ...",Software Engineer
274,"['position', 'description', 'compassion', 'pas...",Software Engineer
60,"['sierra', 'nevada', 'corporation’s', 'electro...",Software Engineer
...,...,...
19,"['mortgage', 'loan', 'originator', 'descriptio...",Loan Officer
454,"['dr', 'horton', 'inc', 'largest', 'homebuilde...",Loan Officer
426,"['dr', 'horton', 'inc', 'largest', 'homebuilde...",Loan Officer
636,"['job', 'descriptionthe', 'auto', 'club', 'gro...",Loan Officer
